In [1]:
import os
import pickle
import torch
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import json
import sys
sys.path.append("/Users/yufanxia/Documents/GitHub")
from perturbgene.data_utils import GeneTokenizer, read_h5ad_file
import scanpy as sc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import itertools

/Users/yufanxia/Documents/GitHub/perturbgene/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
adata = sc.read_h5ad("/Users/yufanxia/Downloads/chunk3562_processed3.h5ad")
#adata.obs["age_group"] = adata.obs["development_stage"].map(age_label_mapping_10).to_list()
adata

/Users/yufanxia/Documents/GitHub/perturbgene/venv/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 10000 × 58604
    obs: 'soma_joinid_column', 'dataset_id', 'assay', 'assay_ontology_term_id', 'cell_type', 'cell_type_ontology_term_id', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'is_primary_data', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_general', 'tissue_general_ontology_term_id', 'raw_sum', 'nnz', 'raw_mean_nnz', 'raw_variance_nnz', 'n_measured_vars', 'batch', 'ZBED1_embedding_status', 'TRAF5_embedding_status', 'GSK3A_embedding_status', 'FBXL19_embedding_status', 'RBX1_embedding_status', 'FBXO7_embedding_status', 'CUL1_embedding_status', 'PINK1_embedding_status', 'PARK7_embedding_status', 'GIGYF2_embedding_status', 'ATP13A2_embedding_status', 'PRKN_embedding_status', 'PLA2G6_embedding_status', 'SKP1_embedding_status', 'LONP2_embedding_status', 'CNTNAP2_embe

In [ ]:
/Users/yufanxia/Downloads/cell_100.npy

In [14]:
target

array(['29-year-old human stage', '6-year-old human stage',
       '49-year-old human stage', '11-month-old human stage',
       '17th week post-fertilization human stage',
       'fifth decade human stage', '67-year-old human stage',
       '44-year-old human stage',
       '17th week post-fertilization human stage',
       'third decade human stage', '78-year-old human stage',
       'Carnegie stage 23', 'sixth decade human stage',
       '17th week post-fertilization human stage',
       '80 year-old and over human stage', '63-year-old human stage',
       '59-year-old human stage', 'ninth decade human stage', 'unknown',
       '80 year-old and over human stage',
       '14th week post-fertilization human stage',
       '15th week post-fertilization human stage',
       '75-year-old human stage', '80 year-old and over human stage',
       '41-year-old human stage', '87-year-old human stage',
       'sixth decade human stage', '1-month-old human stage',
       '14th week post-fertili

In [23]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

# Convert to np.array using toarray() to avoid matrix issues
embeddings = np.asarray(adata.X.toarray())  # shape: (n_samples, embedding_dim)
target = np.array(adata.obs["development_stage"].to_list())  # shape: (n_samples,)

# One-hot encode the categorical labels
encoder = OneHotEncoder(sparse_output=False)
target_onehot = encoder.fit_transform(target.reshape(-1, 1))  # convert target into 2D before encoding

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, target_onehot, test_size=0.2, random_state=42)

# Train a logistic regression model (multi-class)
model = LogisticRegression(max_iter=1000, multi_class='ovr')
model.fit(X_train, np.argmax(y_train, axis=1))  # Using argmax to convert back to label format for logistic regression

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
print(f"Accuracy: {accuracy}")


/Users/yufanxia/Documents/GitHub/perturbgene/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.9155


In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import numpy as np

# Prepare the data
embeddings = np.asarray(adata.X[:100, :].toarray())  # shape: (n_samples, embedding_dim)
target = np.array(adata.obs["development_stage"].to_list()[:100])  # shape: (n_samples,)

# Encode the categorical labels to integers
label_encoder = LabelEncoder()
target_encoded = label_encoder.fit_transform(target)  # Convert to integers

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, target_encoded, test_size=0.2, random_state=42)

# Convert to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)  # For classification, labels should be long integers
y_test = torch.tensor(y_test, dtype=torch.long)

# Define a simple neural network
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Model parameters
input_size = X_train.shape[1]  # Number of embedding dimensions
hidden_size = 64               # You can adjust this
num_classes = len(np.unique(target_encoded))  # Number of unique labels (development stages)

# Create the model
model = SimpleNN(input_size, hidden_size, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the neural network
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model on the test data
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = accuracy_score(y_test, predicted)
    print(accuracy)

Epoch [100/1000], Loss: 0.0000
Epoch [200/1000], Loss: 0.0000
Epoch [300/1000], Loss: 0.0000
Epoch [400/1000], Loss: 0.0000
Epoch [500/1000], Loss: 0.0000
Epoch [600/1000], Loss: 0.0000
Epoch [700/1000], Loss: 0.0000
Epoch [800/1000], Loss: 0.0000
Epoch [900/1000], Loss: 0.0000
Epoch [1000/1000], Loss: 0.0000
0.0
